In [ ]:
!pip -q install lightkurve==2.* numpy pandas scipy astropy tqdm pyarrow

In [ ]:
import numpy as np
import pandas as pd
from typing import Optional, Dict, Any, List, Tuple
from tqdm import tqdm
from astropy.timeseries import BoxLeastSquares
from scipy.signal import savgol_filter
import lightkurve as lk

def _to_clean_arrays(time_in, flux_in):
    t = np.array(time_in, dtype=float)
    f = np.array(flux_in, dtype=float)
    m = np.isfinite(t) & np.isfinite(f)
    t, f = t[m], f[m]
    if len(t) == 0: return t, f
    order = np.argsort(t)
    return t[order], f[order]

def _median_cadence_min(time):
    if len(time) < 2: return 30.0
    dt_days = np.nanmedian(np.diff(time))
    return float(dt_days * 24.0 * 60.0)

def _savgol_detrend(time, flux, window_length=401, polyorder=2):
    if len(flux) < 7: return time, flux
    wl = max(5, int(window_length)); wl = wl if wl % 2 == 1 else wl + 1
    wl = min(wl, len(flux) - (1 - len(flux) % 2))
    if wl < 5: return time, flux
    trend = savgol_filter(flux, window_length=wl, polyorder=polyorder, mode="interp")
    med = np.nanmedian(trend) if np.isfinite(trend).any() else 1.0
    trend = np.where(trend == 0, med, trend)
    return time, flux / trend - 1.0

def _lk_flatten(time, flux, window_length=401, polyorder=2, break_tolerance=5):
    try:
        lc = lk.LightCurve(time=time, flux=flux)
        wl = max(5, int(window_length)); wl = wl if wl % 2 == 1 else wl + 1
        lc_flat = lc.flatten(window_length=wl, polyorder=polyorder, break_tolerance=break_tolerance)
        return np.array(lc_flat.time.value), np.array(lc_flat.flux.value)
    except Exception:
        return _savgol_detrend(time, flux, window_length=window_length, polyorder=polyorder)

def detrend_series(time_in, flux_in,
                   method: str = "lightkurve",
                   window_length: Optional[int] = None,
                   polyorder: int = 2,
                   cadence_min: Optional[float] = None,
                   max_transit_hours: float = 10.0) -> Tuple[np.ndarray, np.ndarray]:
    t, f = _to_clean_arrays(time_in, flux_in)
    if len(t) < 50: return t, f
    if window_length is None:
        if cadence_min is None:
            dt_days = np.nanmedian(np.diff(t)) if len(t) > 1 else 0.0208333
            cadence_min = float(dt_days * 24.0 * 60.0)
        target_minutes = max_transit_hours * 60.0 * 2.5
        window_length = max(51, int(round(target_minutes / max(cadence_min, 1e-6))))
        if window_length % 2 == 0: window_length += 1
    if method == "lightkurve":
        return _lk_flatten(t, f, window_length=window_length, polyorder=polyorder)
    return _savgol_detrend(t, f, window_length=window_length, polyorder=polyorder)

def in_transit_mask(time, period, epoch, duration_days, pad_factor=1.0):
    t = np.asarray(time)
    P = float(period)
    dur = float(duration_days) * pad_factor

    phase = ((t - epoch + 0.5*P) % P) / P - 0.5
    half = 0.5 * dur / P
    return np.abs(phase) <= half

def transit_indices(time, period, epoch):
    t = np.asarray(time)
    k = np.floor((t - epoch)/period + 0.5).astype(int)
    return k

def robust_std(x):
    x = np.asarray(x)
    m = np.isfinite(x)
    if not np.any(m): return np.nan
    mad = np.nanmedian(np.abs(x[m] - np.nanmedian(x[m])))
    return 1.4826 * mad

def odd_even_depth_test(time, flux, period, epoch, duration_days):
    in_mask = in_transit_mask(time, period, epoch, duration_days, pad_factor=1.0)
    if not np.any(in_mask):
        return np.nan, np.nan, np.nan, 0, 0
    k = transit_indices(time, period, epoch)
    k_in = k[in_mask]
    f_in = flux[in_mask]
    odd = (k_in % 2 != 0); even = ~odd
    if np.sum(odd) < 3 or np.sum(even) < 3:
        return np.nan, np.nan, np.nan, int(np.sum(odd)), int(np.sum(even))
    depth_odd = np.nanmedian(f_in[odd]) - np.nanmedian(flux[~in_mask])
    depth_even = np.nanmedian(f_in[even]) - np.nanmedian(flux[~in_mask])
    sigma_out = robust_std(flux[~in_mask])
    sig_odd = sigma_out/np.sqrt(max(1, np.sum(odd)))
    sig_even = sigma_out/np.sqrt(max(1, np.sum(even)))
    z = (depth_odd - depth_even) / np.sqrt(sig_odd**2 + sig_even**2)
    return float(depth_odd), float(depth_even), float(z), int(np.sum(odd)), int(np.sum(even))

def secondary_check(time, flux, period, epoch, duration_days, window_factor=1.0):
    P = float(period); dur = float(duration_days) * window_factor
    epoch2 = epoch + 0.5 * P
    m2 = in_transit_mask(time, P, epoch2, dur, pad_factor=1.0)
    if not np.any(m2): return np.nan, np.nan
    depth2 = np.nanmedian(flux[m2]) - np.nanmedian(flux[~m2])
    sigma_out = robust_std(flux[~m2])
    snr2 = depth2 / (sigma_out / np.sqrt(max(1, np.sum(m2))))
    return float(depth2), float(snr2)

def centroid_shift(time, in_mask, c1, c2):
    if c1 is None or c2 is None:
        return (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)
    c1 = np.asarray(c1); c2 = np.asarray(c2)
    good = np.isfinite(c1) & np.isfinite(c2) & np.isfinite(time)
    if not np.any(good):
        return (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)
    in_mask = in_mask & good
    out_mask = (~in_mask) & good
    if np.sum(in_mask) < 3 or np.sum(out_mask) < 10:
        return (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)
    c1_in, c1_out = np.nanmedian(c1[in_mask]), np.nanmedian(c1[out_mask])
    c2_in, c2_out = np.nanmedian(c2[in_mask]), np.nanmedian(c2[out_mask])
    dc1, dc2 = c1_in - c1_out, c2_in - c2_out
    dmag = np.sqrt(dc1**2 + dc2**2)
    s1 = robust_std(c1[out_mask])/np.sqrt(np.sum(in_mask))
    s2 = robust_std(c2[out_mask])/np.sqrt(np.sum(in_mask))
    z1 = dc1 / (s1 if s1>0 else np.nan)
    z2 = dc2 / (s2 if s2>0 else np.nan)
    zmag = dmag / (np.sqrt((s1**2+s2**2)) if (s1>0 and s2>0) else np.nan)
    return float(dc1), float(dc2), float(dmag), float(z1), float(z2), float(zmag)

def lc_stats(time, flux, in_mask):
    out_mask = ~in_mask
    if np.sum(out_mask) < 10:
        return np.nan, np.nan, int(np.sum(in_mask)), int(np.sum(out_mask))
    f_out = flux[out_mask]
    sigma = robust_std(f_out)
    if not np.isfinite(sigma) or sigma == 0:
        return np.nan, np.nan, int(np.sum(in_mask)), int(np.sum(out_mask))
    outlier_rate = float(np.mean(np.abs(f_out - np.nanmedian(f_out)) > 5*sigma))
    return float(sigma), outlier_rate, int(np.sum(in_mask)), int(np.sum(out_mask))

def sde_from_grid(period_grid, power_grid):
    p = np.asarray(power_grid)
    if p.size < 5 or not np.isfinite(p).any(): return np.nan
    mu, sd = np.nanmean(p), np.nanstd(p)
    if sd == 0 or not np.isfinite(sd): return np.nan
    return float((np.nanmax(p) - mu)/sd)

def sde_local(time, flux, period, duration_days, width_factor=0.5, n_durations=15, oversample=10):
    if len(time) < 100: return np.nan
    P = float(period)
    pmin = max(0.05, P*(1.0 - width_factor))
    pmax = P*(1.0 + width_factor)
    dur_grid = np.linspace(max(0.25*duration_days, 0.5/24.0), min(2*duration_days, 10/24.0), n_durations)
    bls = BoxLeastSquares(time, flux)
    res = bls.autopower(dur_grid, minimum_n_transit=3, frequency_factor=oversample)
    sel = (res.period >= pmin) & (res.period <= pmax) & np.isfinite(res.power)
    if not np.any(sel): return np.nan
    return sde_from_grid(res.period[sel], res.power[sel])

def harmonic_powers(time, flux, period, duration_days):
    try:
        bls = BoxLeastSquares(time, flux)
        resP  = bls.power(period, duration_days, time0=np.nanmin(time)+0.1*period)
        resH1 = bls.power(period/2.0, duration_days/2.0, time0=np.nanmin(time)+0.05*period)
        resH2 = bls.power(period*2.0, duration_days*2.0, time0=np.nanmin(time)+0.2*period)
        pP = np.nanmax(resP.power); pH1 = np.nanmax(resH1.power); pH2 = np.nanmax(resH2.power)
        return float(pH1/pP if pP>0 else np.nan), float(pH2/pP if pP>0 else np.nan)
    except Exception:
        return np.nan, np.nan


In [ ]:
import numpy as np
import pandas as pd
from typing import Optional, Dict, Any, List, Tuple
from tqdm import tqdm

from scipy.signal import savgol_filter
import lightkurve as lk
from google.colab import drive
drive.mount('/content', force_remount=False)

VET_PARQUET = "/content/vetting_kepler.parquet"
CAND_PARQUET = "/content/candidates_bls.parquet"
FEAT_OUT = "/content/candidate_features.parquet"

vet_df = pd.read_parquet(VET_PARQUET)
cand_df = pd.read_parquet(CAND_PARQUET)
print("Loaded vet_df rows:", len(vet_df))
print("Loaded candidates:", len(cand_df))
display(cand_df.head(2))

from typing import Any

def _pick_row_by_obid(vet_idx: pd.DataFrame, obid: Any) -> pd.Series:
    blk = vet_idx.loc[obid]
    if isinstance(blk, pd.DataFrame):
        return blk.iloc[0]
    return blk

def _to_float_array(x):
    if isinstance(x, np.ndarray):
        return x.astype(float)
    if isinstance(x, list):
        return np.array(x, dtype=float)
    if isinstance(x, pd.Series):
        for v in x:
            if isinstance(v, np.ndarray):
                return v.astype(float)
            if isinstance(v, list):
                return np.array(v, dtype=float)

        return np.array(x, dtype=float)
    return np.array(x, dtype=float)

def _safe_centroid_array(val):
    if val is None:
        return None
    if isinstance(val, float) and np.isnan(val):
        return None
    try:
        arr = _to_float_array(val)
        return arr
    except Exception:
        return None

vet_idx = vet_df.set_index("obs_block_id", drop=False)

rows: List[Dict[str, Any]] = []
for i, cand in tqdm(cand_df.iterrows(), total=len(cand_df), desc="Features"):
    obid = cand.get("obs_block_id")
    if (obid is None) or (obid not in vet_idx.index):
        continue
    block = _pick_row_by_obid(vet_idx, obid)
    try:
        t = _to_float_array(block["time"])
        f = _to_float_array(block["flux"])
    except Exception as e:
        print(f"[WARN] could not parse time/flux for {obid}: {repr(e)}")
        continue

    m = np.isfinite(t) & np.isfinite(f)
    t, f = t[m], f[m]
    if t.size < 100:
        continue
    order = np.argsort(t)
    t, f = t[order], f[order]
    f = f - np.nanmedian(f)

    try:
        P   = float(cand["period"])
        t0  = float(cand["epoch"])
        dur = float(cand["duration"])
    except Exception as e:
        print(f"[WARN] bad candidate params for {obid}: {repr(e)}")
        continue

    q = dur / P if P > 0 else np.nan
    n_trans = int(np.floor((np.nanmax(t) - np.nanmin(t)) / P)) if np.isfinite(P) and P > 0 else 0

    in_mask = in_transit_mask(t, P, t0, dur, pad_factor=1.0)

    power = float(cand.get("power", np.nan))
    sde = np.nan
    grid_power = cand.get("bls_power_grid", None)
    grid_period = cand.get("bls_period_grid", None)
    if isinstance(grid_power, list) and len(grid_power) > 5:
        sde = sde_from_grid(grid_period or [], grid_power)
    if not np.isfinite(sde):
        sde = sde_local(t, f, P, dur, width_factor=0.3, n_durations=15, oversample=10)

    d_odd, d_even, z_oe, n_in_odd, n_in_even = odd_even_depth_test(t, f, P, t0, dur)

    sec_depth, sec_snr = secondary_check(t, f, P, t0, dur)


    rms_out, outlier_rate, N_in, N_out = lc_stats(t, f, in_mask)

    mom1 = _safe_centroid_array(block.get("mom_centr1"))
    mom2 = _safe_centroid_array(block.get("mom_centr2"))
    psf1 = _safe_centroid_array(block.get("psf_centr1"))
    psf2 = _safe_centroid_array(block.get("psf_centr2"))
    pos1 = _safe_centroid_array(block.get("pos_corr1"))
    pos2 = _safe_centroid_array(block.get("pos_corr2"))
    cx   = _safe_centroid_array(block.get("cx_tpf"))
    cy   = _safe_centroid_array(block.get("cy_tpf"))

    dc1_mom, dc2_mom, dmag_mom, z1_mom, z2_mom, zmag_mom = centroid_shift(t, in_mask, mom1, mom2)
    dc1_psf, dc2_psf, dmag_psf, z1_psf, z2_psf, zmag_psf = centroid_shift(t, in_mask, psf1, psf2)
    dc1_pos, dc2_pos, dmag_pos, z1_pos, z2_pos, zmag_pos = centroid_shift(t, in_mask, pos1, pos2)
    dc1_tpf, dc2_tpf, dmag_tpf, z1_tpf, z2_tpf, zmag_tpf = centroid_shift(t, in_mask, cx, cy)

    h12, h2 = harmonic_powers(t, f, P, dur)

    rows.append({
        "obs_block_id": obid,
        "target_id": block.get("target_id"),
        "label": block.get("label"),
        "mission": block.get("mission"),
        "sector": block.get("sector"),
        "quarter": block.get("quarter"),
        "campaign": block.get("campaign"),
        "period": P,
        "epoch": t0,
        "duration": dur,
        "duty_cycle": q,
        "bls_power": power,
        "sde": sde,
        "transit_count": n_trans,
        "depth": float(cand.get("depth", np.nan)),
        "depth_err": float(cand.get("depth_err", np.nan)),
        "odd_depth": d_odd,
        "even_depth": d_even,
        "odd_even_z": z_oe,
        "n_in_odd": n_in_odd,
        "n_in_even": n_in_even,
        "sec_depth": sec_depth,
        "sec_snr": sec_snr,
        "rms_out": rms_out,
        "outlier_rate": outlier_rate,
        "n_in": N_in,
        "n_out": N_out,
        "mom_dc1": dc1_mom, "mom_dc2": dc2_mom, "mom_dmag": dmag_mom,
        "mom_z1": z1_mom, "mom_z2": z2_mom, "mom_zmag": zmag_mom,
        "psf_dc1": dc1_psf, "psf_dc2": dc2_psf, "psf_dmag": dmag_psf,
        "psf_z1": z1_psf, "psf_z2": z2_psf, "psf_zmag": zmag_psf,
        "pos_dc1": dc1_pos, "pos_dc2": dc2_pos, "pos_dmag": dmag_pos,
        "pos_z1": z1_pos, "pos_z2": z2_pos, "pos_zmag": zmag_pos,
        "tpf_dc1": dc1_tpf, "tpf_dc2": dc2_tpf, "tpf_dmag": dmag_tpf,
        "tpf_z1": z1_tpf, "tpf_z2": z2_tpf, "tpf_zmag": zmag_tpf,
        "harm_P_over2_ratio": h12,
        "harm_2P_ratio": h2,
    })

features_df = pd.DataFrame.from_records(rows)
print("Feature rows:", len(features_df))
display(features_df.head(3))

features_df.to_parquet(FEAT_OUT, index=False)
print("Saved →", FEAT_OUT)


In [ ]:
np.sum(np.abs(features_df["odd_even_z"].fillna(0)) > 3), len(features_df)

np.sum(features_df["sec_snr"].fillna(0) > 3)

for col in ["mom_zmag", "psf_zmag", "pos_zmag", "tpf_zmag"]:
    if col in features_df:
        print(col, np.sum(np.abs(features_df[col].fillna(0)) > 3))